# Profitable App Profiles for the App Store and Google Play Markets

Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
opened_file_apple = open('AppleStore.csv')
opened_file_android = open('googleplaystore.csv')

from csv import reader
read_file = reader(opened_file_apple)
apple_apps = list(read_file)
read_file = reader(opened_file_android)
android_apps = list(read_file)

del android_apps[10473]

explore_data(apple_apps, 0, 5, True)
explore_data(android_apps, 0, 5, True)


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7198
Number of columns: 16
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Ed

There is a problem with the Google Play data store, there are some duplicate apps. This is cause because there are more than one review updated and they are different rows of the dataset.

To avoid duplicate rows from our data set to improve and be sure we are analyzing the correct data, we should delete duplicate data.

We will do it right now:

In [4]:
duplicate_apps = []
unique_apps = []

for app in android_apps:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


We could remove random duplicated row. But we are going to use some defined criteria.
We are going to leave the row who has the biggest reviewers, because it will be the most recent data.

In [5]:
reviews_max = {}

for app in android_apps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print('Uniques apps: ',len(reviews_max))

android_clean = []
already_added = []

for app in android_apps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
        
explore_data(android_clean, 0, 5, True)

Uniques apps:  9659
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Number of rows: 9659
Number of columns: 13


In [6]:
def isEnglishApp(app_name):
    noEnglishCharCount = 0
    for letter in app_name:
        if ord(letter) > 127:
            noEnglishCharCount += 1
        if noEnglishCharCount > 3:
            return False
    return True

In [7]:
android_english_apps = []
apple_english_apps = []

for android_app in android_clean:
    if isEnglishApp(android_app[0]):
        android_english_apps.append(android_app)
        
for apple_app in apple_apps[1:]:
    if isEnglishApp(apple_app[1]):
        apple_english_apps.append(apple_app)
        
explore_data(apple_english_apps, 0, 5, True)
explore_data(android_english_apps, 0, 5
             , True)
        

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 6183
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'A

In [8]:
def removeLetters(string):
    newString = string.replace('$','')
    
def filter_free_apps(app_list, price_index, currency_symbol=False):
    free_apps = []
    for app in app_list:
        price_string = app[price_index]
        #print(price_string)
        
        if currency_symbol and price_string == 'Free':
            free_apps.append(app)
        elif not currency_symbol:
            price = float(price_string)
            if price == 0.0:
                free_apps.append(app)
            
    return free_apps

free_android_apps = filter_free_apps(android_english_apps, 6, True)
free_apple_apps = filter_free_apps(apple_english_apps, 4)

explore_data(free_apple_apps, 0, 5, True)
explore_data(free_android_apps, 0, 5, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 3222
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'A

Our goal is to find some category that is quite successful on both mobile stores. First, we are going to build an Android app and lets wait for six month to analyze how it performs. If it would be ranked well we will start to build the iOS app.

In [16]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        
def freq_table(dataset, index):
    freq = {}
    for row in dataset:
        key = row[index]
        if key in freq:
            freq[key] += 1
        else:
            freq[key] = 1
        
    return freq
        
print('Apple Store')
display_table(free_apple_apps, 11)
print('\n')
print('----------------------------------------')
print('Category - Android')
display_table(free_android_apps, 1)
print('\n')
print('----------------------------------------')
print('Genres - Android')
display_table(free_android_apps, 9)

Apple Store
Games : 1874
Entertainment : 254
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 84
Utilities : 81
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 51
News : 43
Travel : 40
Finance : 36
Weather : 28
Food & Drink : 26
Reference : 18
Business : 17
Book : 14
Navigation : 6
Medical : 6
Catalogs : 4


----------------------------------------
Category - Android
FAMILY : 1675
GAME : 862
TOOLS : 750
BUSINESS : 407
LIFESTYLE : 346
PRODUCTIVITY : 345
FINANCE : 328
MEDICAL : 313
SPORTS : 301
PERSONALIZATION : 294
COMMUNICATION : 287
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 190
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 124
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 73
WEATHER : 71
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 55
BEAUTY : 53


In [18]:
apple_genres = freq_table(free_apple_apps, 11)
android_genres = freq_table(free_android_apps, 1)

for genre in apple_genres:
    total = 0
    len_genre = 0
    for row_app in free_apple_apps:
        genre_app = row_app[11]
        if genre_app == genre:
            user_rating = float(row_app[5])
            total += user_rating
            len_genre += 1
    average_genre = total / len_genre
    print('Genre ' + genre + ' Average: ' + str(average_genre))

Genre Productivity Average: 21028.410714285714
Genre Education Average: 7003.983050847458
Genre Utilities Average: 18684.456790123455
Genre Sports Average: 23008.898550724636
Genre Health & Fitness Average: 23298.015384615384
Genre Entertainment Average: 14029.830708661417
Genre Medical Average: 612.0
Genre Photo & Video Average: 28441.54375
Genre Finance Average: 31467.944444444445
Genre Catalogs Average: 4004.0
Genre Weather Average: 52279.892857142855
Genre Shopping Average: 26919.690476190477
Genre Lifestyle Average: 16485.764705882353
Genre News Average: 21248.023255813954
Genre Social Networking Average: 71548.34905660378
Genre Book Average: 39758.5
Genre Music Average: 57326.530303030304
Genre Food & Drink Average: 33333.92307692308
Genre Business Average: 7491.117647058823
Genre Travel Average: 28243.8
Genre Games Average: 22788.6696905016
Genre Navigation Average: 86090.33333333333
Genre Reference Average: 74942.11111111111


In [ ]:
def filterCharacters(stringValue):
    return stringValue.replace('+','').replace(',', '')
    
for category in android_genres:
    total = 0
    len_category=0
    for row_app in free_android_apps:
        category_app = row_app[1]
        if category_app == row_app:
            installs = float(filterCharacters(row_app[5])))
            total += installs
            len_category += 1
    
    average_category = total / len_category
    print('Category: ' + category + ' Installs: ' + str(average_category))
            
    